In [406]:
import numpy as np
import matplotlib.pyplot as plt

In [407]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=5000)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
5000
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [408]:
def predict_train(input_str,
                  letters,
                  max_history=4,
                  max_index: int = 4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                # idx_strength = W[prev_inp + str(prev_indices[h_idx])][inp][inp_has_index]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
        else:
            new_index = max(IdxO, key=IdxO.get)
        WIN[inp][new_index] += 1

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)
            HistoryW[h_idx][key_I][inp][new_index] += 1

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW, HistoryHW, IDX

max_index_L1 = 32
max_history_L1 = 8

max_history_L1_ds = 1

HistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1, max_index=max_index_L1)

for h_idx in range(max_history_L1):
    print(f'{h_idx=}')
    for wk, wv in HistoryW[h_idx].items():
        print(wk, wv)


h_idx=0
c0 {'h': {0: 1}}
h0 {'a': {0: 2}}
a0 {'p': {0: 1}, 't': {8: 2}, 'y': {24: 1}, 's': {22: 1}, 'c': {26: 1}, 'b': {12: 1}}
p0 {'t': {0: 1}}
t0 {'e': {0: 1}, 'o': {4: 3}, 'r': {15: 1}, ' ': {17: 3}}
e0 {'r': {0: 3, 20: 1}, ' ': {29: 1, 12: 1, 11: 1, 9: 1, 20: 2}, 's': {21: 2}, 'a': {18: 2}, 'e': {28: 1}}
r0 {' ': {0: 1}, 'h': {20: 1}, 'e': {12: 1}}
 0 {'i': {0: 2}, 'b': {7: 3}, 't': {19: 3}, 'g': {4: 1}, 'k': {3: 1}, 'm': {13: 2}, 'c': {14: 1}, 'd': {7: 1}, 's': {3: 2}, 'u': {2: 1}, 'f': {30: 1}, 'h': {10: 1}, 'y': {9: 1}, 'l': {26: 1}}
i0 {'n': {0: 1}, 'e': {13: 1}, 'm': {18: 1}, ' ': {22: 1}, 's': {18: 1}}
n0 {' ': {1: 1}, 'n': {1: 1, 8: 1}, 'o': {11: 1}, 'd': {30: 2}}
 1 {'w': {0: 2}, 'c': {5: 2}, 'o': {24: 2}, 't': {29: 7}, 'r': {24: 4}, 'g': {2: 2}, 's': {8: 1}, 'u': {3: 1}, 'd': {6: 1}, 'f': {20: 1}, 'a': {29: 4, 18: 1, 20: 1}, 'n': {31: 1}, 'v': {8: 1}, 'j': {5: 1}}
w0 {'h': {1: 1}, ' ': {2: 2}, 'o': {31: 1}}
h1 {'i': {1: 1}, 'r': {8: 2}, 'a': {17: 1}, ' ': {2: 1}}
i1 {'c': 

In [409]:
def predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []
    AL = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            # print(f'{h_idx=}')
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        max_arr = []
        for v in HO.values():
            col_max = max(v, key=v.get)
            max_arr.append(col_max)
        
        max_arr = np.array(max_arr)
        # tf = np.mean(np.abs(max_arr - max_arr[np.newaxis].T))

        tf = length
        # tf = len(HO)

        TF.append(tf)

        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}

            if val[max_idx] > max_val:
                max_val = val[max_idx]
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=}', max_symbol_index, max_val)

        # anomaly
        if max_symbol == next_inp:
            AL.append(0)
        else:
            ni_v = HO.get(next_inp, 0)
            if ni_v == 0:
                AL.append(100)
            else:
                max_ni = max(ni_v, key=ni_v.get)
                AL.append(abs(ni_v[max_ni] - max_val))

        new_indices = [max_symbol_index]

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    TF.append(0)
    AL.append(0)
    return err, TF, AL

err_L1, TF_L1, AL_L1 = predict(input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)
# err_L1, TF_L1_no, AL_L1_no = predict(input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)

print(err_L1, err_L1 / (len(input_str) / 100))

max_symbol='h' next_inp='h' 14 3
max_symbol='e' next_inp='a' 23 12
max_symbol=' ' next_inp='p' 18 9
max_symbol='d' next_inp='t' 23 7
max_symbol=' ' next_inp='e' 15 8
max_symbol='o' next_inp='r' 4 11
max_symbol='p' next_inp=' ' 10 6
max_symbol=' ' next_inp='i' 17 8
max_symbol='n' next_inp='n' 0 7
max_symbol=' ' next_inp=' ' 1 12
max_symbol='w' next_inp='w' 0 12
max_symbol='h' next_inp='h' 1 11
max_symbol='i' next_inp='i' 1 8
max_symbol='c' next_inp='c' 1 8
max_symbol='h' next_inp='h' 2 8
max_symbol=' ' next_inp=' ' 2 8
max_symbol='w' next_inp='w' 1 10
max_symbol='e' next_inp='e' 1 12
max_symbol=' ' next_inp=' ' 3 11
max_symbol='a' next_inp='a' 1 11
max_symbol='r' next_inp='r' 1 9
max_symbol='e' next_inp='e' 2 9
max_symbol=' ' next_inp=' ' 4 16
max_symbol='i' next_inp='i' 2 14
max_symbol='n' next_inp='n' 1 13
max_symbol='t' next_inp='t' 1 14
max_symbol='r' next_inp='r' 2 8
max_symbol='o' next_inp='o' 0 11
max_symbol='d' next_inp='d' 0 8
max_symbol='u' next_inp='u' 0 8
max_symbol='c' next

In [410]:
def get_word_stats(input_str: str | list, TF: list, threshold=0.0):
    buffer: list = [input_str[0]]
    WordStats: dict[str, int] = {}
    WordIndex: dict[str, int] = {}
    next_wrd_idx = 0

    NextLayerOut = []
    DownstreamIndex = {}

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 and inp_idx >= len(input_str) - 1:
            continue

        # if TF[inp_idx] - prev_tf > 0.0: # and abs(TF[inp_idx] - prev_tf) > 0.2:
        # and TF[inp_idx] > threshold:
        if TF[inp_idx] > prev_tf and TF[inp_idx] > TF[inp_idx + 1]:
            word = ''.join(buffer)

            WordStats.setdefault(word, 0)
            WordStats[word] += 1

            if not WordIndex.get(word):
                WordIndex.setdefault(word, next_wrd_idx)
                next_wrd_idx += 1

            wrd_idx = WordIndex.get(word)
            # NextLayerOut.append(str(wrd_idx))
            NextLayerOut.append(word)

            DownstreamIndex.setdefault(inp_idx, 1)

            buffer = [inp]
            prev_tf = TF[inp_idx]

            continue

        TF[inp_idx] = 0

        buffer.append(inp)
        prev_tf = TF[inp_idx]

    # sort
    WordStats = dict(sorted(WordStats.items(), key=lambda item: item[1], reverse=True))

    NextLayerOut.append(str(next_wrd_idx))
    return WordStats, WordIndex, NextLayerOut, DownstreamIndex


TF_L1 = np.array(TF_L1, dtype='float64')
AL_L1 = np.array(AL_L1, dtype='float64')
TF_L1 /= np.max(TF_L1)
AL_L1 /= np.max(AL_L1)
TF_L1 += AL_L1

TF_max_coef = 0.4

# TF_L1[np.where(TF_L1 < np.mean(TF_L1))] = 0
# TF_1 = TF_L1 - np.pad(TF_L1, (1, 0))[:-1]
# TF_2 = TF_L1 - np.pad(TF_L1, (0, 1))[1:]
# TF_L1 = TF_1 + TF_2

# TF_L1[np.where(TF_L1 < 0)] = 0
# TF_L1[np.where(TF_L1 < TF_max_coef)] = 0
TF_L1 = TF_L1.tolist()

WordStats_L1, WordIndex_L1, NextLayerOut_L1, DownstreamIndex_L1 = get_word_stats(input_str=input_str,
                                                                                 TF=TF_L1, 
                                                                                 threshold=0.0, )
print(len(WordStats_L1))
mean_WS = np.mean(np.array(list(WordStats_L1.values())))
print(f'{mean_WS=}')
WS_len = 0
for key, val in WordStats_L1.items():
    # if len(key.strip()) > 1 and val >= mean_WS:
    if val >= mean_WS:
        WS_len += 1
    print(key, val)
print(f'{WS_len=}')

print(f'{NextLayerOut_L1=}')
print(f'{len(NextLayerOut_L1)=}')

print(f'{WordIndex_L1=}')

print(f'{DownstreamIndex_L1=}')


772
mean_WS=2.261658031088083
 a 38
 b 31
 s 26
 h 24
 w 24
 t 23
er 19
 p 19
 f 18
in 18
 i 16
 o 16
 the 15
th 15
nd 14
he 14
 c 14
 he 14
 d 13
on 13
ing 11
 r 11
or 11
 th 11
ng 11
e  10
 y 10
an 10
 g 10
n  9
st 9
 m 9
en 9
ou 9
as 9
es 9
nd  8
ow 8
 n 8
t  8
 l 8
ed 8
 the  8
ooh 7
om 7
it 7
ie 7
is 7
ll 7
ist 6
obi 6
ee 6
aid 6
hat 6
 to 6
ad 6
hr 5
ly 5
ere 5
op 5
and 5
at 5
to 5
id 5
 do 5
 you 5
oo 5
all 5
f  5
ime 5
d  5
g  5
uzz 5
se 5
ot 5
ra 5
lo 5
nie 4
of 4
ome 4
 co 4
re 4
tt 4
ed  4
ut 4
 i  4
ui 4
 wi 4
nn 4
rs 4
et 4
ri 4
la 4
im 4
be 4
 he  4
le 4
nch 4
al 4
ng  4
 in 3
uc 3
oph 3
 kn 3
ai 3
me 3
ea 3
 if 3
the 3
ooh  3
 a  3
so 3
said 3
topher 3
't 3
 q 3
ik 3
ame 3
ry 3
ul 3
lf 3
o  3
 v 3
el 3
r  3
 thi 3
 me 3
zz 3
g no 3
ho 3
ly  3
son 3
ei 3
ar 3
 of 3
ty 3
ood 3
ef 3
ey 3
ab 3
ch 2
tr 2
ees 2
ard 2
nst 2
mp 2
ump 2
 is 2
 as  2
 as 2
ti 2
ls 2
nl 2
uld 2
nt 2
fe 2
n't  2
 be 2
inn 2
wh 2
en i 2
 j 2
ust 2
 go 2
ay 2
him 2
n' 2
ou sa 2
e's 2
now 2
ns  2
ckl 2

In [411]:
def downstream_train(input: list, next_layer_out: list, downstream_index: dict[int, int]):
    DSStats = {}

    ds_idx = 1
    ds_inp = next_layer_out[0]

    for inp_idx, inp in enumerate(input[:-1]):
        has_ds_input = downstream_index.get(inp_idx)
        if has_ds_input:
            ds_inp = next_layer_out[ds_idx]
            ds_idx += 1

        ds_key = ds_inp + ':' + inp
        DSStats.setdefault(ds_key, 0)
        DSStats[ds_key] += 1

    return DSStats


DSStats = downstream_train(input=input_str,
                           next_layer_out=NextLayerOut_L1,
                           downstream_index=DownstreamIndex_L1,)

for wk, wv in DSStats.items():
    print(wk, wv)

cch:c 1
cch:h 1
apt:a 1
apt:p 1
apt:t 1
er:e 19
er:r 19
 i:  16
 i:i 16
n :n 9
n :  9
whi:w 1
whi:h 1
whi:i 1
ch:c 2
ch:h 2
 we :  2
 we :w 1
 we :e 1
are:a 1
are:r 1
are:e 1
 in:  3
 in:i 3
 in:n 3
tr:t 2
tr:r 2
od:o 1
od:d 1
uc:u 3
uc:c 3
ed to win:e 1
ed to win:d 1
ed to win:  2
ed to win:t 1
ed to win:o 1
ed to win:w 1
ed to win:i 1
ed to win:n 1
nie:n 4
nie:i 4
nie:e 4
 the:  15
 the:t 15
 the:h 15
 the:e 15
 p:  19
 p:p 19
ooh:o 14
ooh:h 7
 a:  38
 a:a 38
nd:n 14
nd:d 14
 s:  26
 s:s 26
ome b:o 1
ome b:m 1
ome b:e 1
ome b:  1
ome b:b 1
ees:e 4
ees:s 2
nd :n 8
nd :d 8
nd :  8
th:t 15
th:h 15
e :e 10
e :  10
st:s 9
st:t 9
ories b:o 1
ories b:r 1
ories b:i 1
ories b:e 1
ories b:s 1
ories b:  1
ories b:b 1
egin :e 1
egin :g 1
egin :i 1
egin :n 1
egin :  1
he:h 14
he:e 14
re :r 1
re :e 1
re :  1
is e:i 1
is e:s 1
is e:  1
is e:e 1
dw:d 1
dw:w 1
ard:a 2
ard:r 2
ard:d 2
 bear:  1
 bear:b 1
 bear:e 1
 bear:a 1
 bear:r 1
 c:  14
 c:c 14
om:o 7
om:m 7
ing:i 11
ing:n 11
ing:g 11
 d:  13
 d:

In [412]:
def downstream_predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            upstream_inp: dict[int, list],
            DSStats: dict[str, int],
            ds_input: list,
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    ds_idx = 1
    ds_inp = ds_input[0]

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            # print(f'{key_I=}')
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)
                # print(f'{key_I=}')

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        # tf = length
        tf = len(HO)

        TF.append(tf)

        # ds input
        has_ds_input = upstream_inp.get(input_idx + 1)
        if has_ds_input:
            ds_inp = ds_input[ds_idx]
            ds_idx += 1

        # max symbol
        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}
            max_idx_val = val[max_idx]

            # ds predict
            ds_key = ds_inp + ':' + key
            ds_stats = DSStats.get(ds_key)
            # if ds_stats:
            #     max_idx_val += ds_stats

            if max_idx_val > max_val and ds_stats:
                max_val = max_idx_val
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=} {ds_inp=} {HO=}', max_symbol_index, max_val)

        # print(f'{O.get(max_symbol).keys()=}')
        new_indices = [max_symbol_index]

        if O.get(max_symbol):
            new_indices = list(O.get(max_symbol).keys())            

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    return err, TF

HistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1_ds, max_index=max_index_L1)

# for h_idx in range(max_history_L1_ds):
#     print(f'{h_idx=}')
#     for wk, wv in HistoryW[h_idx].items():
#         print(wk, wv)

err_L1_ds, TF_L1_ds = downstream_predict(input_str=input_str, 
                                        HistoryW=HistoryW, 
                                        upstream_inp=DownstreamIndex_L1,
                                        DSStats=DSStats,
                                        ds_input=NextLayerOut_L1,
                                        max_history=max_history_L1_ds,
                                        max_index=max_index_L1, )

print(f'{err_L1=}', err_L1 / (len(input_str) / 100))
print(f'{err_L1_ds=}', err_L1_ds / (len(input_str) / 100))

max_symbol='h' next_inp='h' ds_inp='cch' HO={'h': {21: 2}, 'o': {21: 3}, 'e': {3: 1}, 'k': {20: 1}, 'i': {13: 1}, 'a': {0: 1}, 'l': {23: 2}, 'r': {27: 1}} 21 2
max_symbol='a' next_inp='a' ds_inp='apt' HO={'a': {20: 3}, 'i': {29: 3}, 'e': {31: 7}, ' ': {8: 3}, 'r': {8: 4}, 'o': {18: 2}, 't': {10: 2}} 20 3
max_symbol='t' next_inp='p' ds_inp='apt' HO={'p': {0: 1}, 'n': {19: 4}, 'b': {16: 2}, 'k': {9: 1}, 'i': {6: 3}, ' ': {11: 3}, 's': {29: 4}, 'r': {11: 2}, 'd': {16: 3}, 't': {30: 2}, 'g': {11: 1}, 'l': {7: 2}, 'w': {29: 1}, 'm': {25: 1}, 'u': {18: 1}} 30 2
max_symbol='t' next_inp='t' ds_inp='apt' HO={'t': {0: 1}, 'l': {15: 1}} 0 1
max_symbol='e' next_inp='e' ds_inp='er' HO={'e': {0: 1}, 'o': {31: 6}, 'h': {17: 2}, ' ': {31: 1}, 'a': {27: 1}, 'w': {14: 1}} 0 1
max_symbol='r' next_inp='r' ds_inp='er' HO={'r': {0: 1}, 'n': {20: 1}, 't': {19: 1}, 'c': {16: 1}, ' ': {5: 8}, 's': {4: 1}, 'd': {1: 1}} 0 1
max_symbol=' ' next_inp=' ' ds_inp=' i' HO={' ': {0: 4}, 'e': {13: 2}} 0 4
max_symbol='i'